In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/Colab Notebooks

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks


In [63]:
import pandas as pd
from datetime import datetime, date
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.tree import export_graphviz
import pydotplus
import re
from six import StringIO
from graphviz import Source

In [22]:
data_hr = pd.read_excel("dataset HR analytics.xlsx", sheet_name = "dataset HR analytics")
data_hr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181 entries, 0 to 180
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Employee_Name               181 non-null    object 
 1   EmpID                       181 non-null    int64  
 2   MarriedID                   181 non-null    object 
 3   MaritalStatusID             181 non-null    object 
 4   GenderID                    181 non-null    object 
 5   EmpStatusID                 181 non-null    object 
 6   RoleID                      181 non-null    object 
 7   LevelID                     181 non-null    object 
 8   PerfScoreID                 181 non-null    object 
 9   Tribe                       181 non-null    object 
 10  Squad                       181 non-null    object 
 11  DOB                         181 non-null    object 
 12  RaceDesc                    181 non-null    object 
 13  DateofHire                  181 non

In [23]:
date_columns = ["DOB", "DateofHire", "DateofTermination", "LastPerformanceReview_Date"]
for i in date_columns:
  data_hr[i] = pd.to_datetime(data_hr[i])
data_hr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181 entries, 0 to 180
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Employee_Name               181 non-null    object        
 1   EmpID                       181 non-null    int64         
 2   MarriedID                   181 non-null    object        
 3   MaritalStatusID             181 non-null    object        
 4   GenderID                    181 non-null    object        
 5   EmpStatusID                 181 non-null    object        
 6   RoleID                      181 non-null    object        
 7   LevelID                     181 non-null    object        
 8   PerfScoreID                 181 non-null    object        
 9   Tribe                       181 non-null    object        
 10  Squad                       181 non-null    object        
 11  DOB                         181 non-null    datetime64[ns]

In [24]:
for i in data_hr.select_dtypes('object').columns:
    print(data_hr[i].value_counts(),'\n')

Ignatius Nicolas Harman    1
Bayu Aji Kurniawan         1
Debby Safira               1
Fajar Lesmana              1
Ester Vanessa              1
                          ..
Enny                       1
Gema Saputera              1
David Suryana Adie         1
Arief Luthfi               1
Bagus                      1
Name: Employee_Name, Length: 181, dtype: int64 

Married        96
Not married    85
Name: MarriedID, dtype: int64 

Single      85
Married     38
Widowed     29
Divorced    29
Name: MaritalStatusID, dtype: int64 

F    105
M     76
Name: GenderID, dtype: int64 

TKP/ Outsource    145
Prohire            36
Name: EmpStatusID, dtype: int64 

Back End Developer          68
Front End Developer         44
Mobile Developer Android    14
Product Owner               11
Business Analyst            10
UI Designer                  9
System Documentation         6
UX Designer                  5
Scrum Master                 3
Mobile Developer iOS         2
DevOps Engineer              

In [25]:
def terminate_status(emp_status):
  if emp_status == "Active":
    return 0
  else:
    return 1
def calculateYear(Date1): 
    today = date.today() 
    age = today.year - Date1.year - ((today.month, today.day) < (Date1.month, Date1.day)) 
    return age

data_hr["Terminate_Status"] = data_hr["EmploymentStatus"].apply(terminate_status)
data_hr["Age"] = data_hr["DOB"].apply(calculateYear)
data_hr["Year_Work"] = data_hr["DateofHire"].apply(calculateYear)

In [26]:
data_hr.head()

,Employee_Name,EmpID,MarriedID,MaritalStatusID,GenderID,EmpStatusID,RoleID,LevelID,PerfScoreID,Tribe,Squad,DOB,RaceDesc,DateofHire,DateofTermination,TermReason,EmploymentStatus,RecruitmentSource,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,LastPerformanceReview_Date,DaysLateLast30,Absences,Terminate_Status,Age,Year_Work
0,A Ridwan,10001,Not married,Single,M,TKP/ Outsource,Product Owner,Senior,Exceed,EWD,Evaluate,1983-10-07,Sunda,2011-05-07,NaT,N/A-StillEmployed,Active,LinkedIn,4.60,5,0,2020-01-17,0,1,0,37,10
1,Ade Setyanto,10002,Not married,Single,M,TKP/ Outsource,Product Owner,Senior,Fully meets,EWD,Pay,1975-05-05,Sunda,2015-03-30,2018-06-16,career change,Voluntarily Terminated,Indeed,4.96,3,6,2017-02-24,0,17,1,46,6
2,Adhitya Bramantya Ranggatyasta,10003,Married,Divorced,F,TKP/ Outsource,Product Owner,Senior,Average,ENT,HunianAja,1988-09-19,Jawa,2011-05-07,2014-09-24,hours,Voluntarily Terminated,LinkedIn,3.02,3,0,2013-05-15,0,3,1,32,10
3,Adji Wijaya Kusuma,10004,Not married,Single,F,TKP/ Outsource,Business Analyst,Middle,Exceed,ENT,Intermoda,1988-09-27,Cina,2008-07-01,NaT,N/A-StillEmployed,Active,Indeed,4.84,5,0,2020-03-01,0,15,0,32,12
4,Agus Andri Putra,10005,Not married,Single,F,Prohire,Business Analyst,Middle,Average,EDU,Squad Pijar Sekolah,1989-08-09,Bali,2011-11-07,2018-06-09,return to school,Voluntarily Terminated,Google Search,5.00,4,0,2017-01-02,0,2,1,31,9


In [27]:
data_hr = data_hr.drop(columns = ["Employee_Name", "EmpID", "MarriedID", "Tribe", "Squad", "RaceDesc", 
                                  "LastPerformanceReview_Date", "DateofHire", "DateofTermination", "TermReason", "EmploymentStatus", 
                                  "RecruitmentSource", "DOB", "RoleID"])

In [28]:
data_hr.head()

,MaritalStatusID,GenderID,EmpStatusID,LevelID,PerfScoreID,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,DaysLateLast30,Absences,Terminate_Status,Age,Year_Work
0,Single,M,TKP/ Outsource,Senior,Exceed,4.60,5,0,0,1,0,37,10
1,Single,M,TKP/ Outsource,Senior,Fully meets,4.96,3,6,0,17,1,46,6
2,Divorced,F,TKP/ Outsource,Senior,Average,3.02,3,0,0,3,1,32,10
3,Single,F,TKP/ Outsource,Middle,Exceed,4.84,5,0,0,15,0,32,12
4,Single,F,Prohire,Middle,Average,5.00,4,0,0,2,1,31,9


In [29]:
data_hr["LevelID"] = data_hr["LevelID"].replace({"Junior": 1, "Middle": 2, "Senior": 3})
data_hr["PerfScoreID"] = data_hr["PerfScoreID"].replace({"Exceed": 5, "Fully meets": 4, "Average": 3, "Need improvement": 2, "Special treatment": 1})
data_hr

,MaritalStatusID,GenderID,EmpStatusID,LevelID,PerfScoreID,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,DaysLateLast30,Absences,Terminate_Status,Age,Year_Work
0,Single,M,TKP/ Outsource,3,5,4.60,5,0,0,1,0,37,10
1,Single,M,TKP/ Outsource,3,4,4.96,3,6,0,17,1,46,6
2,Divorced,F,TKP/ Outsource,3,3,3.02,3,0,0,3,1,32,10
3,Single,F,TKP/ Outsource,2,5,4.84,5,0,0,15,0,32,12
4,Single,F,Prohire,2,3,5.00,4,0,0,2,1,31,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,Single,F,TKP/ Outsource,1,3,4.60,4,0,0,14,0,34,7
177,Married,F,TKP/ Outsource,3,5,5.00,3,0,0,4,0,44,4
178,Single,F,TKP/ Outsource,1,5,5.00,4,0,0,16,1,34,10
179,Single,F,TKP/ Outsource,1,4,4.10,3,0,0,11,0,46,7


In [33]:
enc = OneHotEncoder(handle_unknown='ignore')
data_hr_dummy = pd.DataFrame(enc.fit_transform(data_hr.loc[:, ["MaritalStatusID", "GenderID", "EmpStatusID"]]).toarray())
data_hr_dummy.columns = enc.get_feature_names(["MarritalStatus", "Gender", "EmpStatus"])
data_hr_dummy

,MarritalStatus_Divorced,MarritalStatus_Married,MarritalStatus_Single,MarritalStatus_Widowed,Gender_F,Gender_M,EmpStatus_Prohire,EmpStatus_TKP/ Outsource
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...
176,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
177,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
178,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
179,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [35]:
data_hr_num = data_hr.drop(columns = ["MaritalStatusID", "GenderID", "EmpStatusID"])
data_model = data_hr_num.join(data_hr_dummy)
data_model

,LevelID,PerfScoreID,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,DaysLateLast30,Absences,Terminate_Status,Age,Year_Work,MarritalStatus_Divorced,MarritalStatus_Married,MarritalStatus_Single,MarritalStatus_Widowed,Gender_F,Gender_M,EmpStatus_Prohire,EmpStatus_TKP/ Outsource
0,3,5,4.60,5,0,0,1,0,37,10,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
1,3,4,4.96,3,6,0,17,1,46,6,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
2,3,3,3.02,3,0,0,3,1,32,10,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,2,5,4.84,5,0,0,15,0,32,12,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,2,3,5.00,4,0,0,2,1,31,9,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,1,3,4.60,4,0,0,14,0,34,7,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
177,3,5,5.00,3,0,0,4,0,44,4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
178,1,5,5.00,4,0,0,16,1,34,10,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
179,1,4,4.10,3,0,0,11,0,46,7,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [36]:
data_X = data_model.drop(columns = ["Terminate_Status"])
data_Y = data_model[["Terminate_Status"]]

In [42]:
data_Y.value_counts()

Terminate_Status
0                   127
1                    54
dtype: int64

In [38]:
#Split Training and Test Data
X_train, X_test, Y_train, Y_test = train_test_split(data_X, data_Y, test_size = 0.2, random_state = 0)
print(X_train.shape, X_test.shape)
X_train.head()

(144, 17) (37, 17)


,LevelID,PerfScoreID,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,DaysLateLast30,Absences,Age,Year_Work,MarritalStatus_Divorced,MarritalStatus_Married,MarritalStatus_Single,MarritalStatus_Widowed,Gender_F,Gender_M,EmpStatus_Prohire,EmpStatus_TKP/ Outsource
145,3,3,4.3,3,0,0,1,51,6,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
45,2,5,3.0,5,0,0,17,50,9,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
122,3,1,4.2,4,0,0,13,44,9,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
178,1,5,5.0,4,0,0,16,34,10,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
24,1,1,4.8,5,0,0,5,-46,9,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0


In [55]:
Y_test.value_counts()

Terminate_Status
0                   28
1                    9
dtype: int64

In [45]:
#Logistic Regression
clf = LogisticRegression(random_state=0, solver = 'liblinear', penalty = 'l2')
clf.fit(X_train, Y_train)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
space = dict()
space['solver'] = ['newton-cg', 'lbfgs', 'liblinear']
space['penalty'] = ['none', 'l1', 'l2', 'elasticnet']
search = GridSearchCV(clf, space, scoring='roc_auc', n_jobs=-1, cv=3, verbose = 2)
search.fit(X_train, Y_train)
print(search.best_params_)
Y_pred = search.predict(X_test)
#confusion matrix
print("confusion matrix:") 
print(confusion_matrix(Y_test, Y_pred))

#classification report
print("classification report:")
report = classification_report(Y_test, Y_pred, output_dict = True)
print(classification_report(Y_test, Y_pred))

# Calculating the F1 Score by comparing the actual and predicted values
f_score = f1_score(Y_test ,Y_pred)
print("f1 score test", f_score)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


{'penalty': 'l2', 'solver': 'newton-cg'}
confusion matrix:
[[24  4]
 [ 6  3]]
classification report:
              precision    recall  f1-score   support

           0       0.80      0.86      0.83        28
           1       0.43      0.33      0.38         9

    accuracy                           0.73        37
   macro avg       0.61      0.60      0.60        37
weighted avg       0.71      0.73      0.72        37

f1 score test 0.375


[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    1.3s finished
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [88]:
#Decision Tree
clf = DecisionTreeClassifier(max_depth = 4)
clf.fit(X_train, Y_train)
space = {'max_leaf_nodes': [7, 10, 15, 20], 'min_samples_split': [5, 6, 8, 10]}
gridsearch2 = GridSearchCV(clf, space, scoring='roc_auc', n_jobs=-1, cv=3, verbose = 2)
grid_dt = gridsearch2.fit(X_train, Y_train)
print(gridsearch2.best_params_)
Y_pred = gridsearch2.predict(X_test)

print("confusion matrix:") 
print(confusion_matrix(Y_test, Y_pred))

print("classification report:")
report = classification_report(Y_test, Y_pred, output_dict = True)
print(classification_report(Y_test, Y_pred))

# Calculating the F1 Score by comparing the actual and predicted values
f_score = f1_score(Y_test ,Y_pred)
print("f1 score test", f_score)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


{'max_leaf_nodes': 7, 'min_samples_split': 6}
confusion matrix:
[[17 11]
 [ 2  7]]
classification report:
              precision    recall  f1-score   support

           0       0.89      0.61      0.72        28
           1       0.39      0.78      0.52         9

    accuracy                           0.65        37
   macro avg       0.64      0.69      0.62        37
weighted avg       0.77      0.65      0.67        37

f1 score test 0.5185185185185185


[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:    0.3s finished


In [54]:
#Random Forest
clf = RandomForestClassifier()
clf.fit(X_train, Y_train)
space = {'bootstrap': [True],
'max_depth': [5, 10, 20],
'min_samples_leaf': [2, 3, 4],
'min_samples_split': [2, 4, 6],
'n_estimators': [100, 200, 300]}
gridsearch1 = GridSearchCV(clf, space, scoring='roc_auc', n_jobs=-1, cv=3, verbose = 2)
gridsearch1.fit(X_train, Y_train)
print(gridsearch1.best_params_)
#memprediksi data test
Y_pred = gridsearch1.predict(X_test)

#confusion matrix 
print("confusion matrix:") 
print(confusion_matrix(Y_test, Y_pred))

#classification report
print("classification report:")
report = classification_report(Y_test, Y_pred, output_dict = True)
print(classification_report(Y_test, Y_pred))

#Calculating the F1 Score by comparing the actual and predicted values
f_score = f1_score(Y_test ,Y_pred)
print("f1 score test", f_score)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 3 folds for each of 81 candidates, totalling 243 fits


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:   35.8s


{'bootstrap': True, 'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 4, 'n_estimators': 100}
confusion matrix:
[[28  0]
 [ 9  0]]
classification report:
              precision    recall  f1-score   support

           0       0.76      1.00      0.86        28
           1       0.00      0.00      0.00         9

    accuracy                           0.76        37
   macro avg       0.38      0.50      0.43        37
weighted avg       0.57      0.76      0.65        37

f1 score test 0.0


[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed:   55.4s finished
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [90]:
clf = DecisionTreeClassifier(max_depth = 4, max_leaf_nodes=7, min_samples_split=6)
clf.fit(X_train, Y_train)
dot_datas = StringIO()
export_graphviz(clf, out_file = dot_datas, feature_names = X_train.columns.tolist(), 
class_names = ['Terminated','Not Terminated'], rounded = True, filled = True)
f = pydotplus.graph_from_dot_data(dot_datas.getvalue()).to_string()
with open('tree_modified.dot', 'w') as file:
    file.write(f)
temp = f
s = Source(temp, filename="test.gv", format="png")
s.view()

'test.gv.png'